In [1]:
import numpy as np
import scipy as sp
from scipy import optimize,special,integrate,stats
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib as mpl
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

import copy

import math 

from bin import *

from IPython.display import clear_output
%load_ext autoreload
%autoreload 2

In [2]:
mpl.use("pgf")

mpl.rcParams.update({
    "pgf.texsystem": "lualatex",
    'font.family': 'serif',
    'font.size' : 8,
    'text.usetex': True,
    'pgf.rcfonts': False,
    'pgf.preamble': tex_preamble,
})

In [3]:
Λ = np.hstack([-model_problem_spectrum(100,.8,1e1)[::-1],model_problem_spectrum(300,.8,5e1)])

In [4]:
q = 15
λmin = np.min(np.abs(Λ))**2
λmax = np.max(np.abs(Λ))**2

poles = np.zeros(q)
weights = np.zeros(q)

k2 = λmin/λmax
Kp = sp.special.ellipk(1-k2)

t = 1j*np.arange(1/2,q+1/2)*Kp/q

sn_,cn_,dn_,_ = sp.special.ellipj(np.imag(t),1-k2) # compute real valued functions to transform to what we need

cn = 1/cn_
dn = dn_ / cn_
sn = 1j * sn_ / cn_

poles = np.real(λmin * sn**2)

weights = (2 * Kp * np.sqrt(λmin))/(np.pi*q) * (cn * dn)

In [5]:
A = np.diag(Λ)

n = len(Λ)
b = np.ones(n,dtype=np.double)
b /= np.linalg.norm(b)

r = lambda x: np.sum( weights / (x[:,None]-poles) ,axis=1)
rAb = Λ*r(Λ**2)*b
fAb = np.sign(Λ)*b

K = 120
K_ro = 65
Q,(a_,b_) = lanczos_reorth(A,b,K,reorth=0)
Q_ro,(a_ro_,b_ro_) = lanczos_reorth(A,b,K_ro,reorth=K)

In [6]:
error_OR = np.full(K,np.nan)
error_fOR = np.full(K,np.nan)
error_OR_ro = np.full(K,np.nan)
error_fOR_ro = np.full(K,np.nan)

for i in range(1,K):
    lank_OR = np.zeros(n)
    for l in range(q):
        lan_lm = streaming_banded_rational(n,i,(0,weights[l],0),(1,0,-poles[l]))
        for j in range(i):
            lan_lm.read_stream(Q[:,j],a_[j],b_[j])

        lan_lm.finish_up()      

        lank_OR += lan_lm()

    error_OR[i] = np.linalg.norm(Λ*(rAb - lank_OR))
    error_fOR[i] = np.linalg.norm(Λ*(fAb - lank_OR))
    
for i in range(1,K_ro):
    lank_OR_ro = np.zeros(n)
    for l in range(q):
        lan_lm = streaming_banded_rational(n,i,(0,weights[l],0),(1,0,-poles[l]))
        for j in range(i):
            lan_lm.read_stream(Q_ro[:,j],a_ro_[j],b_ro_[j])

        lan_lm.finish_up()      

        lank_OR_ro += lan_lm()

    error_OR_ro[i] = np.linalg.norm(Λ*(rAb - lank_OR_ro))
    error_fOR_ro[i] = np.linalg.norm(Λ*(fAb - lank_OR_ro))

In [7]:
fig = plt.figure(figsize=(figure_width*cm,figure_height*cm))
ax = fig.add_axes([left, bottom, width, height])

ks = np.arange(K)

ax.plot(ks,error_OR_ro,\
        **line_styles['l4'],markevery=1)
ax.plot(ks,error_fOR_ro,\
        **line_styles['l3'],markevery=1)

ax.plot(ks,error_OR,\
        **line_styles['l2'],markevery=1)
ax.plot(ks,error_fOR,\
        **line_styles['l1'],markevery=1)

ax.set_yscale('log')
plt.grid(True,linestyle=':',linewidth=.5)


#plt.title(f'$\mathbf{{A}}$-norm error bounds: $f(x) = \sqrt{{x}}$, circle contour')
plt.xlabel('number of matvecs: $k$')

plt.savefig('imgs/ch6_sign_rat.pdf')
#plt.close()